# Table of Content

- [Table of Content](#table-of-content)


# 0-General
[Back to Table of Content](#table-of-content)
# NutriJIA – Feasibility, Safety, and Exploratory Clinical Analysis of a Nutritional Intervention

## Introduction
[Back to Table of Content](#table-of-content)

Nutritional interventions in pediatric and adolescent populations are increasingly explored as low-threshold strategies to support health and disease management. Before drawing conclusions about clinical effectiveness, it is essential to establish whether an intervention is **feasible**, **well-adhered to**, and **safe**—especially when biological safety markers and anthropometric parameters may change over time.

NutriJIA evaluates a structured nutritional intervention compared to a control condition with repeated assessments at predefined time points, most prominently:

- baseline visit: `visite_0_wo0`
- post-intervention follow-up: `visite_1_wo12`

In this notebook, feasibility is operationalised primarily through **adherence** metrics. Safety is evaluated using anthropometric and laboratory markers, including:

- weight / BMI
- Vitamin B12
- transferrin saturation
- zinc
- beta-carotene

The analyses focus on within-person change from baseline to post-intervention and on between-group differences (intervention vs. control), typically with baseline adjustment.


## Objective
[Back to Table of Content](#table-of-content)

The primary goal of this analysis is **not** to maximize predictive performance, but to provide a transparent, reproducible evaluation of **feasibility and safety** and to generate **exploratory clinical signals** that can inform future confirmatory studies.

More specifically, we aim to:

1. Construct an analysis-ready participant-level dataset with:
   - group assignment (intervention vs. control)
   - adherence metrics (definition depends on available data)
   - safety endpoints (weight/BMI, Vitamin B12, transferrin saturation, zinc, beta-carotene)
   - key baseline covariates for adjustment (as pre-specified)

2. Perform the **main feasibility and safety analyses**:
   - Quantify adherence and summarise feasibility descriptively (primary).
   - Compare safety markers from baseline to post-intervention between groups using baseline-adjusted models (primary).

3. Perform **exploratory clinical analyses** (secondary):
   - Explore additional outcomes (if available) using the same baseline-adjusted framework.
   - Treat all non-safety endpoints as hypothesis-generating.

4. Integrate results into a coherent interpretation:
   - Emphasise feasibility/safety conclusions and uncertainty.
   - Clearly separate primary (feasibility/safety) from exploratory findings.


## Methodological Role
[Back to Table of Content](#table-of-content)

This notebook is the central, reproducible record of:

- how raw study exports are cleaned and reshaped,
- how adherence, safety endpoints, and covariates are defined, and
- how baseline-adjusted group comparisons are implemented and reported.

Structurally, we distinguish:

1. **Data processing**
   - Import and harmonise visit-level data
   - Derive an analysis-ready dataset (e.g., one row per participant per visit and/or one row per participant with change scores)

2. **Exploratory data analysis**
   - Missingness and distributions
   - Study characteristics and baseline comparability (Table 1-style summary)

3. **Primary analyses: feasibility & safety**
   - Adherence summaries (feasibility)
   - Baseline-adjusted between-group comparisons for safety endpoints

4. **Secondary analyses: exploratory clinical outcomes**
   - Additional baseline-adjusted models (clearly labeled exploratory)
   - Sensitivity analyses (e.g., alternative adherence definitions, missing-data scenarios)

5. **Interpretation**
   - Summary of feasibility and safety evidence
   - Transparent reporting of limitations and implications for future trials

## Acknowledgements
[Back to Table of Content](#table-of-content)

Statistical analysis and data preparation were conducted by **Dr. Steven Ngandeu Schepanski**, who also oversaw the development of this notebook.


# Setup: Imports & Paths
[Back to Table of Content](#table-of-content)

In [16]:
import re
from pathlib import Path

import numpy as np
import pandas as pd

# Define project paths

In [17]:
# The user specified the intended working directory on the local machine
# On your laptop this should exist and will be used for reading and writing
# In other environments the directory may not exist, so we fall back safely
PROJECT_DIR = Path("/Users/stevenschepanski/Documents/04_ANALYSIS/NutriJIA")


In [18]:
# The notebook location is documented here so that anyone reading the code
# understands where it is expected to live in the project structure
NOTEBOOK_DIR = PROJECT_DIR / "scr"

In [19]:
# The data file path inside the project structure
DATA_FILE = PROJECT_DIR / "data" / "NutriJIA_SPSS_Showoff.xlsx"

In [20]:
# If the project directory does not exist, fall back to the current folder
# This makes the script robust when someone runs it in a different environment
if PROJECT_DIR.exists():
    WORK_DIR = PROJECT_DIR
else:
    WORK_DIR = Path.cwd()

print("Working directory used for this run")
print(WORK_DIR)

Working directory used for this run
/Users/stevenschepanski/Documents/04_ANALYSIS/NutriJIA


# Pandas display preferences

In [21]:
# These settings improve readability when inspecting intermediate tables
pd.set_option("display.max_columns", 200)
pd.set_option("display.width", 160)

# Read data

In [22]:
# Read the single sheet Excel file
# If the file contains only one sheet, pandas will read it by default
# We also explicitly convert column names to strings to avoid edge cases
df_long = pd.read_excel(DATA_FILE)
df_long.columns = [str(c) for c in df_long.columns]

print("Raw long data shape")
print(df_long.shape)

print("Raw long data columns")
print(df_long.columns.tolist())

Raw long data shape
(115, 16)
Raw long data columns
['study_id', 'time', 'grp', 'kg', 'kg_percentile', 'kl', 'kl_percentile', 'bmi', 'bmi_percentile', 'globarztvas', 'globpatvas', 'tv_adhrz', 'Transferrin_Sättigung_percent', 'Zink_HP_microgramm_per_l', 'Vitamin_B12_nanogramm_per_l', 'beta_carotin_microgramm_per_l']


# Basic validation of the long format structure

In [23]:
# The expected structure is one row per person per time point
# We verify that the combination of study_id and time is unique
# If it is not unique, pivoting would require aggregation which is risky
key_cols = ["study_id", "time"]

dup_mask = df_long.duplicated(subset=key_cols, keep=False)
if dup_mask.any():
    print("Warning duplicate rows detected for study_id and time")
    print(df_long.loc[dup_mask, key_cols + ["grp"]].sort_values(key_cols).head(20))
else:
    print("No duplicates detected for study_id and time")

print("Number of unique people")
print(df_long["study_id"].nunique())

print("Time point distribution")
print(df_long["time"].value_counts(dropna=False))

No duplicates detected for study_id and time
Number of unique people
23
Time point distribution
time
V0     23
V1     23
V2     23
TV1    23
TV2    23
Name: count, dtype: int64


# Derive a clean group variable from grp

In [24]:
# Functin to extract group assignment from the grp column
# The grp column encodes group assignment using a pattern like
# nJIA-P-0-01  for control
# nJIA-P-1-02  for intervention
#
# There may be minor typos such as missing dashes
# We will extract the first standalone 0 or 1 after the nJIA-P marker
# If extraction fails, group will be missing and we will flag those rows

def extract_group_from_grp(value):
    """
    Extract group indicator from the grp string.

    Expected patterns include
    nJIA-P-0-01
    nJIA-P-1-02

    Also tolerates minor formatting issues such as missing a dash
    nJIA-P1-15

    Returns
    0 for control
    1 for intervention
    np.nan if parsing fails
    """
    if pd.isna(value):
        return np.nan

    s = str(value).strip()

    # First attempt
    # Look for nJIA then P then any separators then capture 0 or 1
    m = re.search(r"nJIA\s*[-_]*\s*P\s*[-_]*\s*([01])", s, flags=re.IGNORECASE)
    if m:
        return int(m.group(1))

    # Second attempt
    # A more permissive fallback that captures any 0 or 1 that is surrounded by separators
    m2 = re.search(r"(?:^|[-_])([01])(?:[-_]|$)", s)
    if m2:
        return int(m2.group(1))

    return np.nan

In [25]:
# Apply the extraction function to create a new group_code column
df_long["group_code"] = df_long["grp"].apply(extract_group_from_grp)

In [26]:
# Map numeric code to human readable labels
# Using explicit labels makes later tables easier to interpret
df_long["group"] = df_long["group_code"].map({0: "control", 1: "intervention"})


In [27]:
# Check whether any rows failed parsing
n_missing_group = df_long["group"].isna().sum()
print("Rows with missing group after parsing")
print(n_missing_group)

if n_missing_group > 0:
    print("Examples of grp values that could not be parsed")
    print(df_long.loc[df_long["group"].isna(), ["study_id", "time", "grp"]].drop_duplicates().head(20))


Rows with missing group after parsing
0


In [28]:
# Ensure each study_id has a single consistent group assignment
# If a study_id appears with both control and intervention, that is a data issue
group_by_id = (
    df_long[["study_id", "group"]]
    .drop_duplicates()
    .groupby("study_id")["group"]
    .nunique(dropna=False)
)

inconsistent_ids = group_by_id[group_by_id > 1].index.tolist()
if len(inconsistent_ids) > 0:
    print("Warning inconsistent group assignment detected for these study_id values")
    print(inconsistent_ids)
    print(df_long.loc[df_long["study_id"].isin(inconsistent_ids), ["study_id", "grp", "group"]].drop_duplicates())
else:
    print("Group assignment appears consistent within study_id")

Group assignment appears consistent within study_id


# Transform long to wide

In [29]:
# Identify value columns
# We keep study_id, time, and group as identifiers and pivot everything else
id_cols = ["study_id", "group", "time", "grp", "group_code"]
value_cols = [c for c in df_long.columns if c not in id_cols]

print("Value columns to be pivoted into wide format")
print(value_cols)

Value columns to be pivoted into wide format
['kg', 'kg_percentile', 'kl', 'kl_percentile', 'bmi', 'bmi_percentile', 'globarztvas', 'globpatvas', 'tv_adhrz', 'Transferrin_Sättigung_percent', 'Zink_HP_microgramm_per_l', 'Vitamin_B12_nanogramm_per_l', 'beta_carotin_microgramm_per_l']


In [30]:
# Pivot to wide format
# This creates one row per study_id and group
# Each measurement becomes a set of time specific columns
# Example resulting column name
# kg_V0
# bmi_V1
df_wide = df_long.pivot_table(
    index=["study_id", "group"],
    columns="time",
    values=value_cols,
    aggfunc="first"
)

In [31]:
# After pivot_table, columns are a MultiIndex with levels
# first level is variable name, second level is time point
# We flatten this into single strings to make the dataset easy to export and use
df_wide.columns = [f"{var}_{time}" for var, time in df_wide.columns]

# Bring study_id and group back as regular columns rather than index
df_wide = df_wide.reset_index()

print("Wide data shape")
print(df_wide.shape)

print("Wide data preview")
print(df_wide.head())

Wide data shape
(23, 40)
Wide data preview
   study_id         group Transferrin_Sättigung_percent_V0 Transferrin_Sättigung_percent_V1 Transferrin_Sättigung_percent_V2 Vitamin_B12_nanogramm_per_l_V0  \
0         1       control                             35.6                             20.5                             13.4                            405   
1         2  intervention                               15                             13.3                                x                            321   
2         3  intervention                             22.8                             26.5                             14.5                            294   
3         4       control                              8.9                              7.5                                x                            380   
4         5  intervention                             17.6                             15.7                              9.5                            294   

  

In [32]:
# Export into the data folder of the project by default
# Using xlsx preserves the wide structure nicely for SPSS demonstration workflows
out_dir = PROJECT_DIR / "data" if PROJECT_DIR.exists() else WORK_DIR
out_file_xlsx = out_dir / "NutriJIA_SPSS_Showoff_wide.xlsx"
out_file_csv = out_dir / "NutriJIA_SPSS_Showoff_wide.csv"

# Write files
df_wide.to_excel(out_file_xlsx, index=False)
df_wide.to_csv(out_file_csv, index=False, encoding="utf-8")

print("Saved wide files")
print(out_file_xlsx)
print(out_file_csv)

Saved wide files
/Users/stevenschepanski/Documents/04_ANALYSIS/NutriJIA/data/NutriJIA_SPSS_Showoff_wide.xlsx
/Users/stevenschepanski/Documents/04_ANALYSIS/NutriJIA/data/NutriJIA_SPSS_Showoff_wide.csv


# Step 1   Inspect unique values per column before cleaning

In [34]:
# Function to summarise unique raw values in each column
# The goal is to understand what non numeric placeholders exist
# We do this before any conversion so we can document what we changed later
# We also keep the output compact by showing only columns that have suspicious values

def summarise_unique_values(series, max_values=25):
    """
    Create a compact summary of unique raw values in a pandas Series

    This is used to detect placeholders like "x", "X", ".", "na", and so on
    We return counts for each distinct value as strings for readability
    """
    value_counts = (
        series.astype(str)
        .str.strip()
        .replace({"nan": np.nan, "None": np.nan})
        .value_counts(dropna=False)
    )

    # Limit output to avoid overwhelming the notebook
    out = value_counts.head(max_values)
    return out

In [35]:
# Identify columns that are not identifiers
# These are the columns where we expect primarily numeric values
id_like_cols = {"study_id", "group"}
data_cols = [c for c in df_wide.columns if c not in id_like_cols]

In [36]:
# Detect columns that likely contain non numeric entries
# We check object dtype columns first as these are the usual suspects
object_cols = [c for c in data_cols if df_wide[c].dtype == "object"]

print("Number of object dtype columns in wide dataset")
print(len(object_cols))

Number of object dtype columns in wide dataset
14


In [37]:
# Create a dictionary of unique value summaries for object columns
# This helps you scan quickly for placeholders and unexpected text
unique_value_summaries = {}
for col in object_cols:
    unique_value_summaries[col] = summarise_unique_values(df_wide[col], max_values=30)


In [38]:
# Print a compact overview of object columns and their top unique values
# This gives you quick insight into which columns need cleaning rules
print("Preview of raw unique values for object dtype columns")
for col, vc in unique_value_summaries.items():
    print("\nColumn name")
    print(col)
    print(vc)

Preview of raw unique values for object dtype columns

Column name
Transferrin_Sättigung_percent_V0
Transferrin_Sättigung_percent_V0
35.6    1
30.1    1
25.2    1
23.7    1
24.4    1
32.5    1
57.4    1
15.5    1
18.7    1
40.6    1
10.6    1
47.4    1
15      1
48.5    1
16.3    1
26.4    1
43.7    1
12.7    1
18.9    1
17.6    1
8.9     1
22.8    1
29.2    1
Name: count, dtype: int64

Column name
Transferrin_Sättigung_percent_V1
Transferrin_Sättigung_percent_V1
20.5    1
31.7    1
35      1
28      1
6.5     1
32.8    1
23.3    1
17.7    1
14.6    1
32.5    1
16.4    1
20.4    1
13.3    1
70.8    1
27.2    1
28.2    1
16.5    1
22.3    1
31.2    1
15.7    1
7.5     1
26.5    1
19.3    1
Name: count, dtype: int64

Column name
Transferrin_Sättigung_percent_V2
Transferrin_Sättigung_percent_V2
x       5
13.4    1
33.5    1
9.4     1
22.5    1
7.4     1
31.7    1
30.8    1
24.1    1
31.1    1
19.5    1
47.2    1
14.9    1
19.1    1
26.1    1
8.7     1
9.5     1
14.5    1
26.8    1
Name: c

# Step 2   Clean data types and harmonise missingness

In [39]:
# We now translate what we saw in the unique value inspection into explicit rules
# Rule set for numeric clinical and laboratory outcomes
# Values are numeric strings
# Placeholder "x" indicates missing information and must become a real missing value
#
# Rule set for adherence variables tv_adhrz_TV1 and tv_adhrz_TV2
# These variables contain true adherence values as numbers
# They also contain group labels as placeholders for not applicable by design
# Therefore "Kontrolle" and "Intervention" are not adherence values and must become missing
# After recoding those labels to missing, the remaining values can be coerced to numeric

# Tokens interpreted as missing values across the dataset
MISSING_TOKENS = {
    "x",
    "X",
    "",
    " ",
    "na",
    "n/a",
    "NA",
    "N/A",
    "nan",
    "NaN",
    ".",
    "-",
    "None",
    "NULL",
    "null",
}

In [40]:
# Tokens that are specific to the adherence variables and indicate not applicable by design
# These must be treated as missing for adherence because they do not encode a score
ADHERENCE_NOT_APPLICABLE_TOKENS = {
    "Kontrolle",
    "Intervention",
    "kontrolle",
    "intervention",
}

In [41]:
# Helper function to clean string columns
def clean_string_tokens(series):
    """
    Clean a Series as text and standardise missing values.

    This helper is used as the first step for all columns
    It does not coerce to numeric
    It only strips whitespace, harmonises decimal commas, and replaces missing tokens
    """
    s = series.astype(str).str.strip()
    s = s.str.replace(",", ".", regex=False)
    s = s.replace(list(MISSING_TOKENS), np.nan)
    return s

In [42]:
# Function to clean and coerce to numeric
def clean_to_numeric(series):
    """
    Clean a Series and coerce it to numeric.

    This should be used for columns that are supposed to be quantitative
    Any non parseable entries become missing values
    """
    s = clean_string_tokens(series)
    return pd.to_numeric(s, errors="coerce")

In [43]:
# Function to clean adherence columns and coerce to numeric
def clean_adherence_to_numeric(series):
    """
    Clean adherence columns and coerce to numeric.

    Special handling
    Group label placeholders are treated as not applicable
    Those placeholders become missing
    Remaining values are coerced to numeric
    """
    s = clean_string_tokens(series)

    # Replace not applicable placeholders with missing
    s = s.replace(list(ADHERENCE_NOT_APPLICABLE_TOKENS), np.nan)

    # Coerce to numeric
    return pd.to_numeric(s, errors="coerce")

In [44]:
# Create a cleaned copy of the wide dataset
# We keep df_wide as the untouched raw wide dataset for auditing and reproducibility
df_wide_clean = df_wide.copy()

In [45]:
# Identify adherence columns by prefix
tv_adhrz_cols = [c for c in df_wide_clean.columns if c.lower().startswith("tv_adhrz")]


In [46]:
# Identify identifier columns that should not be converted
id_cols_clean = ["study_id", "group"]

In [47]:
# Identify all remaining columns as candidates for numeric conversion
# We will treat adherence columns with their specialised cleaner
candidate_numeric_cols = [
    c for c in df_wide_clean.columns
    if c not in id_cols_clean and c not in tv_adhrz_cols
]

print("Candidate numeric columns")
print(candidate_numeric_cols)

print("Adherence columns")
print(tv_adhrz_cols)

Candidate numeric columns
['Transferrin_Sättigung_percent_V0', 'Transferrin_Sättigung_percent_V1', 'Transferrin_Sättigung_percent_V2', 'Vitamin_B12_nanogramm_per_l_V0', 'Vitamin_B12_nanogramm_per_l_V1', 'Vitamin_B12_nanogramm_per_l_V2', 'Zink_HP_microgramm_per_l_V0', 'Zink_HP_microgramm_per_l_V1', 'Zink_HP_microgramm_per_l_V2', 'beta_carotin_microgramm_per_l_V0', 'beta_carotin_microgramm_per_l_V1', 'beta_carotin_microgramm_per_l_V2', 'bmi_V0', 'bmi_V1', 'bmi_V2', 'bmi_percentile_V0', 'bmi_percentile_V1', 'bmi_percentile_V2', 'globarztvas_V0', 'globarztvas_V1', 'globarztvas_V2', 'globpatvas_V0', 'globpatvas_V1', 'globpatvas_V2', 'kg_V0', 'kg_V1', 'kg_V2', 'kg_percentile_V0', 'kg_percentile_V1', 'kg_percentile_V2', 'kl_V0', 'kl_V1', 'kl_V2', 'kl_percentile_V0', 'kl_percentile_V1', 'kl_percentile_V2']
Adherence columns
['tv_adhrz_TV1', 'tv_adhrz_TV2']


In [48]:
# Apply numeric cleaning to standard quantitative columns
conversion_log = []
for col in candidate_numeric_cols:
    before_non_missing = df_wide_clean[col].notna().sum()
    df_wide_clean[col] = clean_to_numeric(df_wide_clean[col])
    after_non_missing = df_wide_clean[col].notna().sum()

    conversion_log.append(
        {
            "column": col,
            "non_missing_before": int(before_non_missing),
            "non_missing_after": int(after_non_missing),
            "new_missing_created": int(before_non_missing - after_non_missing),
        }
    )

In [49]:
# Apply specialised cleaning to adherence columns
# The conversion log is extended so we can diagnose if unexpected information is lost
for col in tv_adhrz_cols:
    before_non_missing = df_wide_clean[col].notna().sum()
    df_wide_clean[col] = clean_adherence_to_numeric(df_wide_clean[col])
    after_non_missing = df_wide_clean[col].notna().sum()

    conversion_log.append(
        {
            "column": col,
            "non_missing_before": int(before_non_missing),
            "non_missing_after": int(after_non_missing),
            "new_missing_created": int(before_non_missing - after_non_missing),
        }
    )

conversion_log_df = (
    pd.DataFrame(conversion_log)
    .sort_values(["new_missing_created", "column"], ascending=[False, True])
)

print("Summary of numeric conversion and missingness changes")
print(conversion_log_df)

print("Cleaned wide data shape")
print(df_wide_clean.shape)

print("Cleaned wide data preview")
print(df_wide_clean.head())

print("Cleaned wide data types preview")
print(df_wide_clean.dtypes.head(30))

Summary of numeric conversion and missingness changes
                              column  non_missing_before  non_missing_after  new_missing_created
37                      tv_adhrz_TV2                  21                  9                   12
36                      tv_adhrz_TV1                  21                 10                   11
11  beta_carotin_microgramm_per_l_V2                  23                 16                    7
8        Zink_HP_microgramm_per_l_V2                  23                 17                    6
2   Transferrin_Sättigung_percent_V2                  23                 18                    5
5     Vitamin_B12_nanogramm_per_l_V2                  23                 19                    4
4     Vitamin_B12_nanogramm_per_l_V1                  23                 22                    1
6        Zink_HP_microgramm_per_l_V0                  23                 22                    1
10  beta_carotin_microgramm_per_l_V1                  23                 

# Step 2a   Quick validation checks for adherence logic

In [50]:
# These checks are designed to confirm your interpretation of the protocol
# We expect adherence at TV1 to be mostly available in the intervention group
# We expect adherence at TV2 to be mostly available in the control group
#
# Because the dataset is small, we summarise counts of non missing by group

if len(tv_adhrz_cols) > 0:
    for col in tv_adhrz_cols:
        print("\nNon missing adherence counts by group for column")
        print(col)
        tmp = (
            df_wide_clean
            .groupby("group")[col]
            .apply(lambda x: x.notna().sum())
            .reset_index(name="n_non_missing")
        )
        print(tmp)

        print("Basic adherence summary by group for column")
        print(col)
        tmp2 = (
            df_wide_clean
            .groupby("group")[col]
            .agg(
                n="count",
                mean="mean",
                sd="std",
                median="median",
                min="min",
                max="max",
            )
            .reset_index()
        )
        print(tmp2)


Non missing adherence counts by group for column
tv_adhrz_TV1
          group  n_non_missing
0       control              0
1  intervention             10
Basic adherence summary by group for column
tv_adhrz_TV1
          group   n  mean      sd  median  min  max
0       control   0   NaN     NaN     NaN  NaN  NaN
1  intervention  10  2.85  1.6675    2.75  1.0  5.0

Non missing adherence counts by group for column
tv_adhrz_TV2
          group  n_non_missing
0       control              9
1  intervention              0
Basic adherence summary by group for column
tv_adhrz_TV2
          group  n      mean        sd  median  min  max
0       control  9  2.777778  1.715938     3.0  0.0  5.0
1  intervention  0       NaN       NaN     NaN  NaN  NaN


# Step 3   Create analysis ready datasets for primary outcomes

In [51]:
# Primary outcomes are globpatvas and globarztvas
# Baseline is V0 and endpoint is V2
# We create explicit baseline and endpoint variables to avoid mistakes later

PRIMARY_OUTCOMES = ["globpatvas", "globarztvas"]
BASELINE_TP = "V0"
ENDPOINT_TP = "V2"

In [52]:
# Function to construct wide column names
def wide_name(var, tp):
    """
    Construct the expected wide column name given a variable and time point
    Example globpatvas_V0
    """
    return f"{var}_{tp}"

In [53]:
# Build a list of required columns for primary outcome analyses
required_primary_cols = ["study_id", "group"]
for var in PRIMARY_OUTCOMES:
    required_primary_cols.append(wide_name(var, BASELINE_TP))
    required_primary_cols.append(wide_name(var, ENDPOINT_TP))

In [54]:
# Verify that all required primary outcome columns are present
missing_primary_cols = [c for c in required_primary_cols if c not in df_wide_clean.columns]
if len(missing_primary_cols) > 0:
    print("Error primary outcome columns missing from cleaned dataset")
    print(missing_primary_cols)
    raise KeyError("Primary outcome columns not found in df_wide_clean")


In [55]:
# Create a primary outcomes analysis dataset
df_primary = df_wide_clean[required_primary_cols].copy()

df_primary = df_primary.rename(
    columns={
        wide_name("globpatvas", BASELINE_TP): "globpatvas_baseline",
        wide_name("globpatvas", ENDPOINT_TP): "globpatvas_endpoint",
        wide_name("globarztvas", BASELINE_TP): "globarztvas_baseline",
        wide_name("globarztvas", ENDPOINT_TP): "globarztvas_endpoint",
    }
)

print("Primary outcomes analysis dataset preview")
print(df_primary.head())

Primary outcomes analysis dataset preview
   study_id         group  globpatvas_baseline  globpatvas_endpoint  globarztvas_baseline  globarztvas_endpoint
0         1       control                  2.0                  2.0                   0.0                   0.0
1         2  intervention                  0.0                  1.0                   1.5                   1.5
2         3  intervention                  1.0                  1.0                   0.0                   0.0
3         4       control                  1.0                  NaN                   0.5                   NaN
4         5  intervention                  2.0                  2.0                   2.0                   1.5


In [56]:
# Simple completeness flags
# These are useful for reporting and for deciding which analysis set is used later
df_primary["complete_globpatvas"] = df_primary[["globpatvas_baseline", "globpatvas_endpoint"]].notna().all(axis=1)
df_primary["complete_globarztvas"] = df_primary[["globarztvas_baseline", "globarztvas_endpoint"]].notna().all(axis=1)

print("Completeness counts for primary outcomes")
print(df_primary[["complete_globpatvas", "complete_globarztvas"]].sum())


Completeness counts for primary outcomes
complete_globpatvas     21
complete_globarztvas    21
dtype: int64


# Step 4   Descriptive baseline tables stratified by group

In [57]:
# Baseline summaries are descriptive only
# They are not inferential tests
# With very small n, these summaries should be interpreted cautiously

def describe_by_group(df, value_col):
    """
    Produce descriptive statistics for a single variable stratified by group
    """
    return (
        df
        .groupby("group")[value_col]
        .agg(
            n="count",
            mean="mean",
            sd="std",
            median="median",
            min="min",
            max="max",
        )
        .reset_index()
    )

In [58]:
# Baseline descriptive tables
print("Baseline descriptive table for globpatvas")
print(describe_by_group(df_primary, "globpatvas_baseline"))

print("Baseline descriptive table for globarztvas")
print(describe_by_group(df_primary, "globarztvas_baseline"))

Baseline descriptive table for globpatvas
          group   n      mean        sd  median  min  max
0       control  11  2.772727  1.941181     2.0  1.0  7.0
1  intervention  12  1.375000  1.720531     1.0  0.0  6.0
Baseline descriptive table for globarztvas
          group   n      mean        sd  median  min  max
0       control  11  0.636364  1.026911     0.5  0.0  3.5
1  intervention  12  0.583333  0.925235     0.0  0.0  2.5
